In [4]:
!curl http://localhost:8000/v1/completions -H "Content-Type: application/json" -d '{"model": "casperhansen/mixtral-instruct-awq","prompt": "San Francisco is a","max_tokens": 100,"temperature": 0}'

{"id":"cmpl-175eb9aaa4a34a30ab8f427b215c7daa","object":"text_completion","created":1712667371,"model":"casperhansen/mixtral-instruct-awq","choices":[{"index":0,"text":" city that is known for its vibrant culture, diverse population, and stunning architecture. It is also a city that is home to some of the most beautiful parks and gardens in the country. If you are planning a trip to San Francisco, be sure to include a visit to one of these stunning gardens in your itinerary.\n\n1. The Japanese Tea Garden: Located in Golden Gate Park, the Japanese Tea Garden is a peaceful oasis in the heart of the city.","logprobs":null,"finish_reason":"length","stop_reason":null}],"usage":{"prompt_tokens":5,"total_tokens":105,"completion_tokens":100}}

In [1]:
import requests

response = requests.post("http://localhost:8000/v1/completions", json={"model": "MaziyarPanahi/WizardLM-2-8x22B-AWQ","prompt": "San Francisco is a","max_tokens": 100,"temperature": 0})

response.text

'{"id":"cmpl-568cb6aa7bf34b39997f925314e91da7","object":"text_completion","created":1713447845,"model":"MaziyarPanahi/WizardLM-2-8x22B-AWQ","choices":[{"index":0,"text":" city that is known for its vibrant culture, stunning architecture, and rich history. It is also a city that is known for its commitment to sustainability and green living. In recent years, San Francisco has become a leader in the green building movement, with a number of innovative and sustainable buildings popping up throughout the city.\\n\\nOne of the most notable examples of green building in San Francisco is the California Academy of Sciences, located in Golden Gate Park. This state-of-the-","logprobs":null,"finish_reason":"length","stop_reason":null}],"usage":{"prompt_tokens":5,"total_tokens":105,"completion_tokens":100}}'

In [ ]:
new_data = []
for key in data:
    for i, time in enumerate(data[key]):
        new_data.append({'nb_tokens': nb_tokens[i], 'time': time, 'key': key})

In [1]:
import numpy as np
import plotly.express as px
import pandas as pd

nb_tokens = [31, 63, 119, 296, 480, 822, 2193]

data_mixtral_8x7b = {
    1: [3.1, 3.2, 3.6, 3.4, 3.5, 3.5, 4.1],
    2: [3.3, 3.3, 3.5, 3.5, 3.8, 3.8, 4.7],
    4: [3.5, 3.6, 3.5, 4.3, 4.1, 4.6, 6.2],
    8: [3.8, 3.8, 4.0, 4.3, 4.9, 5.7, 9.3],
    16: [4.3, 4.6, 4.9, 6.0, 6.7, 8.5, 15.5],
    32: [6.0, 6.4, 7.6, 8.7, 10.9, 14.2, np.nan],
    64: [10.0, 10.5, 11.6, 15.5, np.nan, np.nan, np.nan],
    128: [18.5, 19.6, 21.5, np.nan, np.nan, np.nan, np.nan],
}

data_wizard2_8x22b = {
    1: [6.0, 6.0, 6.3, 6.8, 7.0, 7.6, 10.9],
    2: [7.2, 7.1, 7.4, 8.4, 8.7, 10.3, 16.7],
    4: [8.0, 8.1, 8.7, 10.3, 11.9, 14.2, 21.3],
    8: [9.0, 9.4, 10.0, 13.0, 16.7, 19.4, 36.2],
    16: [11.0, 12.2, 13.2, 21.1, 26.5, 31.9, 66.4],
    32: [16.0, 17.6, 22.6, 33.6, 37.7, 56.7, np.nan],
    64: [28.0, 31.8, 35.9, 56.0, 71.7, np.nan, np.nan],
    128: [52.0, 55.3, 67.0, 111.7, np.nan, np.nan, np.nan],
}

data = data_mixtral_8x7b

new_data = []
for key in data:
    for i, time in enumerate(data[key]):
        new_data.append({'nb_tokens': nb_tokens[i], 'time': time, 'nb_users': key})

df = pd.DataFrame(new_data)
df['total'] = df.nb_users * 100 / df.time
df['per_user'] = df.total / df.nb_users
df.head()

,nb_tokens,time,nb_users,total,per_user
0,31,3.1,1,32.258065,32.258065
1,63,3.2,1,31.250000,31.250000
2,119,3.6,1,27.777778,27.777778
3,296,3.4,1,29.411765,29.411765
4,480,3.5,1,28.571429,28.571429


In [3]:
fig = px.line(df, x='nb_tokens', y='time', color="nb_users", log_x=True, title="Time to generate 100 tokens based on prompt size and number of concurrent users")
fig.update_traces(mode='lines+markers')
fig.update_layout(
    xaxis = {
        'tickmode': 'array',
        'tickvals': nb_tokens,
        'title': 'Size of the prompt (number of tokens)',
    },
    yaxis = {
        'title': 'Time (seconds)',
    },
    legend = {
        'title': 'Number of concurrent users',
    }
)

In [7]:
fig = px.line(df, x='nb_tokens', y='total', color="nb_users", log_x=True, range_y=[0, 250], title="Nombre total de tokens générés par seconde en fonction de la taille du prompt et du nombre d'utilisateurs simultanés.")
fig.update_traces(mode='lines+markers')
fig.update_layout(
    xaxis = {
        'tickmode': 'array',
        'tickvals': nb_tokens,
        'title': 'Taille du prompt (nombre de tokens)',
    },
    yaxis = {
        'title': 'Nombre total de tokens générés par seconde',
    },
    legend = {
        'title': 'Nombre d\'utilisateurs simultanés',
    }
)

In [5]:
fig = px.line(df, x='nb_tokens', y='per_user', color="nb_users", log_x=True, title="Nombre de tokens générés par seconde et par utilisateur en fonction de la taille du prompt et du nombre d'utilisateurs simultanés.")
fig.update_traces(mode='lines+markers')
fig.update_layout(
    xaxis = {
        'tickmode': 'array',
        'tickvals': nb_tokens,
        'title': 'Taille du prompt (nombre de tokens)',
    },
    yaxis = {
        'title': 'Nombre de tokens générés par seconde et par utilisateur',
    },
    legend = {
        'title': 'Nombre d\'utilisateurs simultanés',
    }
)